In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from random import randint

In [2]:
df = pd.read_csv('links of nike and adidas.csv')
li=[]
df

,Model Name,Stockx Link,Ebay Link,Unnamed: 3
0,Nike+A4:A92 Dunk Low Brazil (2020),https://stockx.com/nike-dunk-low-brazil-2020,https://www.ebay.com/itm/2020-Nike-Dunk-Low-Br...,NaN
1,Nike Air Max 90 Green Camo,https://stockx.com/nike-air-max-90-duck-camo-volt,https://www.ebay.com/itm/Nike-Air-Max-90-SP-Gh...,NaN
2,Nike Air Max 90 Reverse Duck Camo (2020),https://stockx.com/nike-air-max-90-reverse-duc...,https://www.ebay.com/itm/NIKE-AIR-MAX-90-SP-AT...,NaN
3,Nike LD Waffle Sacai White Nylon,https://stockx.com/nike-ld-waffle-sacai-white-...,https://www.ebay.com/itm/Nike-x-Sacai-LD-Waffl...,NaN
4,Nike Air Max 2 Light Atmos,https://stockx.com/nike-air-max-2-light-atmos,https://www.ebay.com/itm/Mens-Nike-X-Atmos-Air...,NaN
5,Nike Air Force 1 Low Supreme Black,https://stockx.com/nike-air-force-1-low-suprem...,https://www.ebay.com/itm/Nike-Air-Force-One-1-...,NaN
6,Nike Dunk Low SP Syracuse (2020),https://stockx.com/nike-dunk-low-sp-syracuse,https://www.ebay.com/itm/Nike-Dunk-Low-SP-Syra...,NaN
7,Nike React Element 87 Anthracite Black,https://stockx.com/nike-react-element-87-anthr...,https://www.ebay.com/itm/Nike-React-Element-87...,NaN
8,Nike Blazer Mid 77 Vintage White Black,https://stockx.com/nike-blazer-mid-77-vintage-...,https://www.ebay.com/itm/Nike-Blazer-Mid-Vinta...,NaN
9,Nike SB Dunk Low Raygun Tie-Dye Black,https://stockx.com/nike-sb-dunk-low-raygun-tie...,https://www.ebay.com/itm/Nike-SB-Dunk-Low-Rayg...,NaN


In [3]:
features = ['Model Name', 'Condition', 'Price (US $)' , 'Brand', 'Type', 'Style' , 'Product Line', 'Color', 'Department', 'Seller Name' ,'Seller Description','Followers', 'Member Since','Feedback #1 (Item as described)', 'Feedback #2 (Communication)', 'Feedback #3 (Shipping time)', 'Feedback #4 (Shipping charges)',  'Feedback Score', 'Percentage Positive Feedback']

In [4]:
i =108
for url in df['Ebay Link'][ : 200]:
    di={}
    try:
        page = requests.get(url)
    except:
        time.sleep(15)
        page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    di['Model Name'] = str(i+1) + '.' + df['Model Name'][i]
    try:
        c = soup.find_all(itemprop = "itemCondition")[0].get_text().replace('\n' , '').replace('\t' , '').strip(' ')
        if(len(c) >0):
            di['Condition'] = c
    except:
        pass
    try:
        price = soup.find( 'span' ,class_ = "notranslate").get_text().replace('\n' , '').replace('\t' , '').strip(' ')
        if(len(price)):
            di['Price (US $)'] = price
    except:
        pass
    
    try:
        trs = soup.find('div', class_='section').find_all('tr')
        check = ['Brand', 'Model Number', 'Type', 'Style' , 'Product Line', 'Color', 'Department']
        for tr in trs:
            tds = tr.find_all('td')
            for j in range(len(tds)):
                if (j%2 == 0):
                    try:
                        field = tds[j].get_text().replace('\n' , '').replace('\t' , '').strip(' ').replace(':' , "")
                        if(field == " "):
                            field = None
                    except:
                        field = None
                else:
                    try:
                        field_value = tds[j].get_text().replace('\n' , '').replace('\t' , '').strip(' ')
                        if field in check:
                            di[field] = field_value
                    except:
                        field_value = None
    except:
        pass
    try:
        fb = soup.find('span', class_ = 'mbg-l').find('a').get_text().replace('\n' , '').replace('\t' , '').strip(' ').replace(':' , "")
        if(len(fb)):
            di['Feedback Score'] = fb
    except:
        di['Feedback Score'] = None
    
    try:
        pfb = soup.find('div', id='si-fb').get_text().replace('\n' , '').replace('\t' , '').strip(' ').replace(':' , "")
        pfb = pfb[ : pfb.find('%')]
        if(len(pfb)):
            di['Percentage Positive Feedback'] = pfb
    except:
        di['Percentage Positive Feedback'] = None
    
    try:
        seller_name = soup.find('span' , class_ ="mbg-nw").get_text().replace('\n' , '').replace('\t' , '').strip(' ').replace(':' , "")
        if(len(seller_name)):
            di['Seller Name'] = seller_name
            seller_link = soup.find('div' , class_ = "mbg vi-VR-margBtm3").find('a')['href']
            try:
                html = BeautifulSoup(requests.get(seller_link).content, 'html.parser')
            except:
                time.sleep(10)
                html = BeautifulSoup(requests.get(seller_link).content, 'html.parser')

            di['Seller Description'] = html.find('div', class_= 'cntnt').find('h2').get_text().replace('\n', '').replace('\t' , '').replace('\r', '').strip(' ')

            c=1
            for h in html.find('div' , id = 'dsr').find_all('div' , class_= 'fl each'):
                f = h.find('span' , class_ ="dsr_type").get_text()
                di['Feedback #' + str(c) + ' (' + f + ')'] = h.find('span' , class_ ="dsr_count bold fl").get_text()
                c+=1
            try:
                s = html.find('div' , id= 'member_info').find_all('span', class_ = 'hide countElem')[0]['contentstring']
                di['Followers'] = s[s.find('>') + 1 : s.find('<' , 2)]
            except:
                di['Followers'] = None
            try:
                st = html.find('div' , id= 'member_info').find('span', class_ = 'info').get_text()
                di['Member Since'] = st
            except:
                di['Member Since'] = None
    except:
        di['Seller Name'] = None
        
    
    
    li.append(di)
    i+=1
    print(i)

109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207


KeyError: 207

In [5]:
df2 = pd.DataFrame(li , columns = features)

In [6]:
df2

,Model Name,Condition,Price (US $),Brand,Type,Style,Product Line,Color,Department,Seller Name,Seller Description,Followers,Member Since,Feedback #1 (Item as described),Feedback #2 (Communication),Feedback #3 (Shipping time),Feedback #4 (Shipping charges),Feedback Score,Percentage Positive Feedback
0,109.ADIDAS YEEZY BOOST 380 MIST,New with box,US $369.90,Nike,Athletic,Athletic Sneakers,Nike Dunk Low,Multi-Color,Men,hotnessfeet215,Welcome to HotnessFeet215's eBay auctions. Maj...,"22,159","Mar 18, 2008","1,073","1,072","1,114","1,128",33160,99.9
1,110.ADIDAS YEEZY BOOST 350 V2 TAIL LIGHT,New with box,US $219.90,Nike,NaN,Athletic Sneakers,NaN,Multi-Color,NaN,hotnessfeet215,Welcome to HotnessFeet215's eBay auctions. Maj...,"22,159","Mar 18, 2008","1,073","1,072","1,114","1,128",33160,99.9
2,111.ADIDAS YEEZY BOOST 350 V2 EARTH,New with box,US $359.99,Nike,Athletic,Running Shoes,NaN,Multicolor,NaN,thecoolshoeshine,"Based in United States, thecoolshoeshine has b...","16,762","Jan 24, 2002","1,911","1,923","1,996","1,990",51176,99.8
3,112.ADIDAS YEEZY 700 V3 ALVAH,New with box,US $450.00,Nike,Athletic,NaN,NaN,Black,Men,None,style junkie and collector of random things,NaN,NaN,NaN,NaN,NaN,NaN,629,100
4,113.ADIDAS YEEZY BOOST 350 V2 SULFUR,New without box,US $144.99,Nike,Athletic,NaN,Nike Air,Multicolor,Men,ragall23,"Based in United States, ragall23 has been an e...",6,"Mar 24, 2019",49,48,49,49,65,None
5,114.ADIDAS YEEZY BOOST 700 WAVE RUNNER SOLID GREY,New with box,US $129.90,Nike,Athletic,Athletic Sneakers,Air Force One,Black,Men,sneakzilla,Follow us on IG: @sneakzilla_official. Sneaker...,"7,549","Jun 21, 2005",852,843,887,904,8373,99.9
6,115.ADIDAS YEEZY 500 STONE,New with box,US $499.99,Nike,Athletic,Skateboarding Shoes,NaN,White,Men,rubberandcotton,"Based in United States, rubberandcotton has be...",458,"Mar 14, 2017",210,213,214,208,1454,None
7,116.ADIDAS YEEZY BOOST 350 V2 CLOUD WHITE (NON...,New with box,US $109.99,Nike,Athletic,Athletic Sneakers,Nike React,Multicolor,Men,jayhoodisback,"Based in United States, jayhoodisback has been...",239,"Jan 24, 2004",337,343,345,354,1390,100
8,117.ADIDAS YEEZY BOOST 700 ANALOG,New with box,US $196.85,Nike,NaN,Athletic Sneakers,Nike Blazer,Multi-Color,NaN,obas818,"Based in United States, obas818 has been an eB...",10,"May 20, 2006",100,103,107,107,187,100
9,118.ADIDAS YEEZY 700 MAUVE,New with box,US $388.00,Raygun x Nike,Athletic,Sneaker,SB,Black/Orange Flash,Men,soles888,"Based in United States, soles888 has been an e...",910,"Sep 05, 2012",646,648,669,673,2952,99.4


In [7]:
df2.to_csv('Red Table New.csv')